In [8]:
# Dependencies
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn import metrics
# Make results reproducible
seed = 1234
np.random.seed(seed)
tf.set_random_seed(seed)

# Loading the dataset
dat = pd.read_csv('Path Of the Data')
values = list(dat.columns.values)
X=dat.drop(['Y_class'],axis=1)


In [10]:
y = dat['Y_class']
y = pd.get_dummies(y, columns=['Y_class']) # One Hot Encoding

y = np.array(y, dtype='float32')
X = np.array(X, dtype='float32')

# Shuffle Data
indices = np.random.choice(len(X), len(X), replace=False)
X_values = X[indices]
y_values = y[indices]

# Creating a Train and a Test Dataset
test_size = 12000 # change accordingly
X_test = X_values[-test_size:]
X_tr = X_values[:-test_size]
y_test = y_values[-test_size:]
y_tr = y_values[:-test_size]
y_t = np.argmax(y_tr, axis=1)

# As the data is imbalance we apply respective balancing technique  

sm = RandomOverSampler(ratio=1.0)
X_train, y_train = sm.fit_sample(X_tr, y_t)
pp = pd.get_dummies(y_train) # One Hot Encoding
y_train = np.array(pp, dtype='float32')
X_train = np.array(X_train, dtype='float32')
X.shape

# Train model


/home/naman/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)


(49941, 7)

In [12]:

graph = tf.Graph()
with graph.as_default():
    # Session

    # Interval / Epochs
    interval = 100 # Change accordingly
    epoch = 500

    # Initialize placeholders
    X_data = tf.placeholder(shape=[None,no_of_features], dtype=tf.float32)
    y_target = tf.placeholder(shape=[None,2], dtype=tf.float32)
    P=tf.placeholder(shape=[None],dtype=tf.int32)
    Y=tf.placeholder(shape=[None],dtype=tf.int32)

    # Input neurons : 29    
    # Output neurons : 2
    hidden_layer_nodes2 = 400
    hidden_layer_nodes1 = 1200

    # Create variables for Neural Network layers
    w1 = tf.Variable(tf.random_normal(shape=[no_of_features,hidden_layer_nodes1])) # Inputs -> Hidden Layer
    b1 = tf.Variable(tf.random_normal(shape=[hidden_layer_nodes1]))   # First Bias
    w2 = tf.Variable(tf.random_normal(shape=[hidden_layer_nodes1,hidden_layer_nodes2])) # Inputs -> Hidden Layer
    b2 = tf.Variable(tf.random_normal(shape=[hidden_layer_nodes2]))  
    w3 = tf.Variable(tf.random_normal(shape=[hidden_layer_nodes2, hidden_layer_nodes2])) # Hidden layer -> Outputs
    b3 = tf.Variable(tf.random_normal(shape=[2]))   # Second Bias
   
    # Operations
       
    logits1 = tf.matmul(X_data, w1,name="LOGIT1") + b1
    hidden_output1=tf.nn.tanh(logits1)
    logits2 = tf.matmul(hidden_output1, w2,name="LOGIT2") + b2
    hidden_output2=tf.nn.tanh(logits2)
    logits3 = tf.matmul(hidden_output2, w3,name="LOGIT3") + b3
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_target, logits=logits3),name='loss')
    
    regularizer = tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2) + tf.nn.l2_loss(w3)
    loss = tf.reduce_mean(loss + 0.01 * regularizer)
    
    # Cost Function
    #loss = tf.reduce_mean(-tf.reduce_sum(y_target * tf.log(final_output), axis=0))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
    #train predict
    logit1 = tf.matmul(X_train, w1,name="LOGIT1") + b1
    rel1=tf.nn.tanh(logit1)
    logit2 = tf.matmul(rel1, w2,name="LOGIT2") + b2
    rel2=tf.nn.tanh(logit2)
    logit3 = tf.matmul(rel2, w3,name="LOGIT2") + b3
    train_prediction = tf.nn.softmax(logit3)
    confusion_matrix_tp= tf.confusion_matrix(tf.argmax(y_train,1),tf.argmax(train_prediction,1),2)

    #Test
    logit1 = tf.matmul(X_test, w1,name="LOGIT1") + b1
    rel1=tf.nn.tanh(logit1)
    logit2 = tf.matmul(rel1, w2,name="LOGIT2") + b2
    rel2=tf.nn.tanh(logit2)
    logit3 = tf.matmul(rel2, w3,name="LOGIT2") + b3
    test_prediction = tf.nn.softmax(logit3)

    confusion_matrix_tf = tf.confusion_matrix(tf.argmax(y_test,1),tf.argmax(test_prediction,1),2)




In [13]:
num_steps=20 #Change
def accuracy(predictions, labels):
  P=np.argmax(predictions,1) 
  Y=np.argmax(labels,1)
  return (100.0 * np.sum(np.argmax(predictions,1) == np.argmax(labels,1))    / predictions.shape[0])
def accuracy1(pred):
        test_predict1 = np.argmax(pred,1)
        return test_predict1

with tf.Session(graph=graph) as session:

    # Initialize variables
    init = tf.global_variables_initializer()
    session.run(init)
    for step in range(num_steps):
        # Training
        print('Training the model... Step :',step)
        for i in range(1, (epoch + 1)):
            session.run(optimizer, feed_dict={X_data: X_train, y_target: y_train})
            if i % interval == 0:
                print('Epoch', i, '|', 'Loss:', session.run(loss, feed_dict={X_data: X_train, y_target: y_train}))
    #print()
    #for i in range(len(X_test)):
        #print('Actual:', y_test[i], 'Predicted:', np.rint(session.run(final_output, feed_dict={X_data: [X_test[i]]})))
    print("Train accuracy: {:.5f}".format(accuracy(train_prediction.eval(), y_train)))
    cm_tr = confusion_matrix_tp.eval()
    print(cm_tr)
    print("Test accuracy: {:.5f}".format(accuracy(test_prediction.eval(), y_test)))
    cm_te = confusion_matrix_tf.eval()
    y_p=accuracy1(test_prediction.eval())
    print(cm_te)
   

Training the model... Step : 0
Epoch 100 | Loss: 802.57
Epoch 200 | Loss: 695.4319
Epoch 300 | Loss: 602.18585
Epoch 400 | Loss: 521.15594
Epoch 500 | Loss: 450.8693
Training the model... Step : 1
Epoch 100 | Loss: 389.891
Epoch 200 | Loss: 336.82483
Epoch 300 | Loss: 290.7504
Epoch 400 | Loss: 250.8034
Epoch 500 | Loss: 216.09802
Training the model... Step : 2
Epoch 100 | Loss: 186.0571
Epoch 200 | Loss: 160.10742
Epoch 300 | Loss: 137.59148
Epoch 400 | Loss: 118.05076
Epoch 500 | Loss: 101.20099
Training the model... Step : 3
Epoch 100 | Loss: 86.56081
Epoch 200 | Loss: 73.98009
Epoch 300 | Loss: 63.101093
Epoch 400 | Loss: 53.740368
Epoch 500 | Loss: 45.67689
Training the model... Step : 4
Epoch 100 | Loss: 38.79979
Epoch 200 | Loss: 32.87468
Epoch 300 | Loss: 27.834486
Epoch 400 | Loss: 23.537336
Epoch 500 | Loss: 19.930746
Training the model... Step : 5
Epoch 100 | Loss: 16.772242
Epoch 200 | Loss: 14.126692
Epoch 300 | Loss: 11.90414
Epoch 400 | Loss: 10.036345
Epoch 500 | Loss: 

ValueError: Expected 2D array, got 1D array instead:
array=[1 0 1 ... 1 0 1].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.